In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import cv2
%load_ext autoreload
%autoreload 2

a = np.load('pose_result__data_paper_180918_MDN_PR_Fly1_001_SG1_behData_images.pkl')
data = a['points3d'].copy()
print('The dimension is ')
print(data.shape)
t = np.linspace(0,9,900)

The dimension is 
(899, 38, 3)


In [2]:
def rotaMatr(v1,v2):
    u = np.cross(v1,v2)
    u = u/np.linalg.norm(u)
    th = intersection_angle(v1,v2)
    r = np.zeros((3,3))
    c = np.cos(th)
    s = np.sin(th)
    ux = u[0]
    uy = u[1]
    uz = u[2]
    r[0,0] = c + ux**2*(1-c)
    r[0,1] = ux*uy*(1-c) - uz*s
    r[0,2] = ux*uz*(1-c) + uy*s
    
    r[1,0] = ux*uy*(1-c) + uz*s
    r[1,1] = c + uy**2*(1-c)
    r[1,2] = uy*uz*(1-c) - ux*s
    
    r[2,0] = uz*ux*(1-c) - uy*s
    r[2,1] = uz*uy*(1-c) + ux*s
    r[2,2] = c + uz**2*(1-c)
    return r
def leg_inner(data, t, leg, pos):   
    '''pos = 1,2,3  t = 0 , leg = 0, 5, 10,       19, 24, 29'''
    ex_point = data[t,leg+pos,:]
    pre_point = data[t,leg+pos-1,:]
    next_point = data[t,leg+pos+1,:]
    pre_v = ex_point - pre_point
    nxt_v = next_point - ex_point
    return intersection_angle(pre_v, nxt_v)
def intersection_angle(a,b):
    return np.arccos(a@b/np.linalg.norm(a)/np.linalg.norm(b))
def ag2(angle):
    return angle/np.pi*180
def set_coord():
    v_ref = np.array([0,0,1])
    v1 = data[0,0,:] - data[0,19,:]
    v2 = data[0,5,:] - data[0,24,:]
    v3 = data[0,10,:] - data[0,29,:]
    v4 = data[0,15,:] - data[0,34,:]
    print(ag2(intersection_angle(v_ref,v1)))
    print(ag2(intersection_angle(v_ref,v2)))
    print(ag2(intersection_angle(v_ref,v3)))
    print(ag2(intersection_angle(v_ref,v4)))
    xx = v1+v2+v3+v4
    x_ref = xx/np.linalg.norm(xx)
    print(x_ref)
    x_ref[1] = 0
    print(x_ref)
    print('x-axis angel: ',ag2(intersection_angle(v_ref,x_ref)))
    return x_ref
coord_x = set_coord()
#an = 38/180*np.pi
#x_ref = np.array([np.sin(an),0,np.cos(an)])
#print(x_ref)

#coord_x = x_ref
coord_y = np.array([0,-1,0])
coord_z = np.cross(coord_x,coord_y)
print('left-coordinate x :', coord_x)
print('left-coordinate y :', coord_y)
print('left-coordinate z :', coord_z)

39.719173687676225
37.766569550937106
40.96779343467542
38.55254876567842
[0.62771434 0.07004704 0.77528583]
[0.62771434 0.         0.77528583]
x-axis angel:  38.99554730142681
left-coordinate x : [0.62771434 0.         0.77528583]
left-coordinate y : [ 0 -1  0]
left-coordinate z : [ 0.77528583  0.         -0.62771434]


In [3]:
angle_data1 = np.zeros((899,18))
angle_data3 = np.zeros((899,18))
def leg_1dof(t):
    i = 0
    for leg in [0,5,10,19,24,29]:
        #if(leg == 0 ): print('Left leg')
        #elif(leg == 19 ): print('Right leg')
        for pos in range(1,4):
            if(pos == 2):
                angle_data1[t,i+pos-1] = -leg_inner(data,t=t,leg=leg,pos=pos)
                #print(ag2(angle_data1[0,i+pos-1]))
            else:
                angle_data1[t,i+pos-1] = leg_inner(data,t=t,leg=leg,pos=pos)
                #print(ag2(angle_data1[0,i+pos-1]))
        i+=3
        #print('\n')
for i in range(899):        
    leg_1dof(i)
t = 0
print('angles of three 1-dof joint: ')
legname = ['left front', 'left middle', 'left hind', 'right front', 'right middle', 'right hind']
for i in range(6):
    print(legname[i])
    print([ag2(angle_data1[t,i*3+j]) for j in range(3)])

angles of three 1-dof joint: 
left front
[117.63232953482924, -80.821096359871, 25.553272540020448]
left middle
[122.18831338730779, -107.15928822681015, 36.98481160025102]
left hind
[106.0468270952644, -99.98323384477206, 25.910724214640414]
right front
[149.4852259735869, -127.57293061849138, 37.52917551926949]
right middle
[123.258058023241, -116.51690971376023, 41.38393744859637]
right hind
[125.4432181135452, -125.15896017476622, 39.35937201066465]


In [8]:
def leg_3dof(t):
    j = 0
    for i in [0,5,10,19,24,29]:
        if(i == 0 or i == 5 or i ==10):
            lrcoord_z = coord_z;
            lrcoord_x = coord_x;
        else:
            lrcoord_z = -coord_z;
            lrcoord_x = -coord_x;
        lrcoord_y = coord_y 
        x_r = data[t,i+1,:] - data[t,i+0,:]
        xr_proj = x_r.copy()
        xr_y = xr_proj[1]
        xr_proj[1] = 0;
        
        theta_z = intersection_angle(lrcoord_z, xr_proj)
        theta_x = intersection_angle(lrcoord_x, xr_proj)
        if(theta_z < np.pi/2):
            angle_1 = -theta_x
        else:
            angle_1 = theta_x
        #print('angle1')
        angle_data3[t,j] = angle_1
        #print(ag2(angle_1))
        angle_2 = np.pi/2 - intersection_angle(lrcoord_y,x_r)
        #print('angle2')
        angle_data3[t,j+1] = angle_2 
        #print(ag2(angle_2))
        y_slope = data[t,i+2,:] - data[t,i+1,:]
        #  how to make sure y_slope direction, Yes, sure
        z_r = np.cross(x_r, y_slope)
        R1 = rotaMatr(lrcoord_x, xr_proj)
        rel_coord_z = R1 @ lrcoord_z
        R2 = rotaMatr(xr_proj, x_r)
        rel_coord_y = R2 @ lrcoord_y
        theta3_y = intersection_angle(rel_coord_y, z_r)
        theta3_z = intersection_angle(rel_coord_z, z_r)
        #print("theta3_y = ",ag2(theta3_y),"theta3_z = ",ag2(theta3_z))
        #print("sum = ",ag2(theta3_y+theta3_z), 'dif = ',ag2(theta3_y-theta3_z))
        if(theta3_y < np.pi/2):
            angle_3 = -theta3_z #+ dif
        else:
            angle_3 = theta3_z #+ dif
        #print('angle3')
        angle_data3[t,j+2] = angle_3
        #print(ag2(angle_3))
        #print('\n')
        j += 3
    return angle_1,angle_2,angle_3

for i in range(899):
    leg_3dof(i)

print('angles of root 3-dof joint: ')
legname = ['left front', 'left middle', 'left hind', 'right front', 'right middle', 'right hind']
for i in range(6):
    print(legname[i])
    print([ag2(angle_data3[t,i*3+j]) for j in range(3)])
np.savetxt('angle_data1.txt',angle_data1,fmt='%.4f')
#np.savetxt('angle_data3.txt',angle_data3,fmt='%.4f')
#np.save('angle_3d.npy',angle_data3)
#np.save('angle_1d.npy',angle_data1)

angles of root 3-dof joint: 
left front
[104.05387503380325, -63.04905775148335, 149.5353063568902]
left middle
[164.33947696603283, -82.76834568737672, 144.93936360292219]
left hind
[99.8174981421469, -76.86026152694096, 49.60149747563612]
right front
[-129.0794804818497, -50.555637147894124, 177.32200284530325]
right middle
[-158.10426581949253, -81.36573525047217, -143.08286323197302]
right hind
[-117.54054350667504, -75.58345524070519, -74.36654920393458]


In [7]:
angle_data1.shape

(899, 18)